## Preprocessing

In [49]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [50]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)

In [51]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

In [52]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [53]:
# Choose a cutoff value and create a list of application types to be replaced
cutoff_value = 500
application_types_to_replace = pd.Series(unique_values.loc["APPLICATION_TYPE"])
application_types_to_replace = application_types_to_replace[application_types_to_replace < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [54]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [55]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts_greater_than_1 = classification_counts[classification_counts > 1]
print(classification_counts_greater_than_1) 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [56]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_values = 500
classifications_to_replace = classification_counts[classification_counts < cutoff_values].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [57]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_encoded = pd.get_dummies(application_df)

In [58]:
# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [59]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()
input_features = X.shape[1]
hidden_layer1_nodes = 128
hidden_layer2_nodes = 60


# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1_nodes, input_dim=input_features, activation='relu'))


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2_nodes, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               6784      
                                                                 
 dense_13 (Dense)            (None, 60)                7740      
                                                                 
 dense_14 (Dense)            (None, 1)                 61        
                                                                 
Total params: 14585 (56.97 KB)
Trainable params: 14585 (56.97 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [62]:
# Train the model
epochs = 100
history = nn.fit(X_train_scaled, y_train, epochs=epochs, shuffle=True, validation_data=(X_test_scaled, y_test))


Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5697 - accuracy: 0.7214 - val_loss: 0.5639 - val_accuracy: 0.7214
Epoch 2/100
804/804 [==============================] - 1s 960us/step - loss: 0.5536 - accuracy: 0.7304 - val_loss: 0.5608 - val_accuracy: 0.7236
Epoch 3/100
804/804 [==============================] - 1s 975us/step - loss: 0.5515 - accuracy: 0.7324 - val_loss: 0.5571 - val_accuracy: 0.7207
Epoch 4/100
804/804 [==============================] - 1s 964us/step - loss: 0.5491 - accuracy: 0.7341 - val_loss: 0.5559 - val_accuracy: 0.7241
Epoch 5/100
804/804 [==============================] - 1s 951us/step - loss: 0.5476 - accuracy: 0.7337 - val_loss: 0.5584 - val_accuracy: 0.7207
Epoch 6/100
804/804 [==============================] - 1s 965us/step - loss: 0.5471 - accuracy: 0.7354 - val_loss: 0.5537 - val_accuracy: 0.7215
Epoch 7/100
804/804 [==============================] - 1s 942us/step - loss: 0.5456 - accuracy: 0.7348 - val_loss: 0.5544 - val_accu

In [63]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5673 - accuracy: 0.7258 - 160ms/epoch - 597us/step
Loss: 0.5672561526298523, Accuracy: 0.7258309125900269


In [64]:
# Export our model to HDF5 file
hdf5_filename = 'AlphabetSoupCharity.h5'

# Save model to file
nn.save(hdf5_filename)

c:\Users\Julian\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [66]:
Overview of the analysis: Explain the purpose of this analysis - To predict whether applicants will be successful if funded by Alphabet Soup.

Results: Using bulleted lists and images to support your answers, address the following questions:

Data Preprocessing

What variable(s) are the target(s) for your model? 
Is__successful__ column is the target for the model.

What variable(s) are the features for your model? 
Features that are used for the model are: 

APPLICATION_TYPE—Alphabet Soup application type
AFFILIATION—Affiliated sector of industry
CLASSIFICATION—Government organization classification
USE_CASE—Use case for funding
ORGANIZATION—Organization type
STATUS—Active status
INCOME_AMT—Income classification
SPECIAL_CONSIDERATIONS—Special considerations for application
ASK_AMT—Funding amount requested
IS_SUCCESSFUL—Was the money used effectively

What variable(s) should be removed from the input data because they are neither targets nor features?
SPECICAL_CONSIDERATIONS column should be removed from the input data because it is neither target nor feature.

How many neurons, layers, and activation functions did you select for your neural network model, and why?
Total number of neurons are 128 + 60 + 1 = 189
Total number of layers are 3
Activation function for hidden layers is relu and for output layer is sigmoid.
These were chosen because relu is good for hidden layers and sigmoid is good for output layer. Also I'm not good at anything more complex at this point.
Were you able to achieve the target model performance?
No, I was not able to achieve the target model performance.
What steps did you take in your attempts to increase model performance?
I tried to increase the number of epochs to 1000 and also tried to add more neurons and layers but it did not help.
Summary: Summarize the overall results of the deep learning model. Include a recommendation for how a different model could solve this classification problem, and then explain your recommendation.
For the overall results I would say that the model is not very accurate. It is only 72% accurate. I would recommend to try a different model like Random Forest Classifier or Logistic Regression.
I think these models would be better for this classification problem.

SyntaxError: invalid character '—' (U+2014) (2560163221.py, line 13)